# Imports e bibliotecas

In [1]:
from keras.datasets import mnist
from keras import layers
from keras import models
from keras.utils import to_categorical


# Carregando e Preprocessando os Dados


In [2]:
# Carregamento dos dados
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000,28,28,1))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000,28,28,1))
test_images = test_images.astype('float32')/255


11490434/11490434 [==============================] - 0s 0us/step


# Transformando os rótulos ( labels ) em categorias


In [3]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


# Exemplo ( Apagar dps)

In [4]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape = (28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64,activation = 'relu'))
model.add(layers.Dense(10, activation= 'softmax'))

# Rede Convolucional Simples

Primeiro teste simples sobre redes convolucionais. A arquitetura da rede inclui uma camada convolucional, uma de flattening e densas.

## Arquitetura da Rede


1. **Primeira Camada Convolucional**: Aplicamos 32 filtros de tamanho 3x3 com função de ativação ReLU.
2. **Flatten**: Convertendo a saída da camada convolucional em um vetor 1D.
3. **Primeira Camada Densa**: Contém 250 neurônios com função de ativação sigmoide.
4. **Camada de Saída**: Contém 10 neurônios (um para cada classe) com função de ativação softmax.

In [5]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.Flatten())
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 flatten_1 (Flatten)         (None, 21632)             0         
                                                                 
 dense_2 (Dense)             (None, 250)               5408250   
                                                                 
 dense_3 (Dense)             (None, 10)                2510      
                                                                 
Total params: 5411080 (20.64 MB)
Trainable params: 5411080 (20.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [7]:
model.fit(train_images, train_labels, epochs=5, batch_size = 64)

Epoch 1/5
938/938 [==============================] - 13s 7ms/step - loss: 0.1607 - accuracy: 0.9508
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0456 - accuracy: 0.9859
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0249 - accuracy: 0.9925
Epoch 4/5
938/938 [==============================] - 4s 5ms/step - loss: 0.0131 - accuracy: 0.9957
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0069 - accuracy: 0.9979


In [8]:
model.evaluate(test_images, test_labels)[1]

313/313 [==============================] - 1s 3ms/step - loss: 0.0527 - accuracy: 0.9850


0.9850000143051147

In [9]:
def treinar(modelo):
    modelo.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

    modelo.fit(train_images, train_labels, epochs=5, batch_size = 64)

    return modelo

In [10]:
def stats(modelo):
    return modelo.evaluate(test_images, test_labels)[1]  # Index 1 for accuracy


In [11]:
#def treinamento(primeiro_layer,epocas,ultimo_layer):
 # model = Model([inpx], ultimo_layer)
  #model.compile(optimizer=keras.optimizers.Adadelta(),
   #     loss=keras.losses.categorical_crossentropy,
    #    metrics=['accuracy'])

 # model.fit(x_train, y_train, epochs=epocas, batch_size=500)

In [12]:
#def stats(x):
 # score = x.evaluate(x_test, y_test, verbose=0)
  #print('loss=', score[0])
  #print('accuracy=', score[1])

# Testes com diferentes tamanhos de filtro

### Teste 1 (5x5)

#### Arquitetura da Rede


1. **Primeira Camada Convolucional**: Aplicamos 32 filtros de tamanho 5x5 com função de ativação ReLU.
2. **Flatten**: Convertendo a saída da camada convolucional em um vetor 1D.
3. **Primeira Camada Densa**: Contém 250 neurônios com função de ativação sigmoide.
4. **Camada de Saída**: Contém 10 neurônios (um para cada classe) com função de ativação softmax.

In [13]:
model5x5 = models.Sequential()
model5x5.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(28, 28, 1)))
model5x5.add(layers.Flatten())
model5x5.add(layers.Dense(250, activation='relu'))
model5x5.add(layers.Dense(10, activation='softmax'))

model5x5.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 24, 24, 32)        832       
                                                                 
 flatten_2 (Flatten)         (None, 18432)             0         
                                                                 
 dense_4 (Dense)             (None, 250)               4608250   
                                                                 
 dense_5 (Dense)             (None, 10)                2510      
                                                                 
Total params: 4611592 (17.59 MB)
Trainable params: 4611592 (17.59 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
treinar(model5x5)


Epoch 1/5
938/938 [==============================] - 5s 5ms/step - loss: 0.1441 - accuracy: 0.9562
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0436 - accuracy: 0.9862
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0257 - accuracy: 0.9919
Epoch 4/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0148 - accuracy: 0.9952
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0097 - accuracy: 0.9969


In [15]:
stats(model5x5)

313/313 [==============================] - 1s 2ms/step - loss: 0.0439 - accuracy: 0.9889


0.9889000058174133

# Teste 7x7


In [16]:
model7x7 = models.Sequential()
model7x7.add(layers.Conv2D(32, (7, 7), activation='relu', input_shape=(28, 28, 1)))
model7x7.add(layers.Flatten())
model7x7.add(layers.Dense(250, activation='relu'))
model7x7.add(layers.Dense(10, activation='softmax'))

model7x7.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 22, 22, 32)        1600      
                                                                 
 flatten_3 (Flatten)         (None, 15488)             0         
                                                                 
 dense_6 (Dense)             (None, 250)               3872250   
                                                                 
 dense_7 (Dense)             (None, 10)                2510      
                                                                 
Total params: 3876360 (14.79 MB)
Trainable params: 3876360 (14.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
treinar(model7x7)

Epoch 1/5
938/938 [==============================] - 5s 4ms/step - loss: 0.1324 - accuracy: 0.9593
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0408 - accuracy: 0.9875
Epoch 3/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0234 - accuracy: 0.9924
Epoch 4/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0137 - accuracy: 0.9956
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0083 - accuracy: 0.9974


In [18]:
stats(model7x7)

313/313 [==============================] - 1s 3ms/step - loss: 0.0382 - accuracy: 0.9896


0.9896000027656555


# Teste 25x25


In [19]:
model25x25 = models.Sequential()
model25x25.add(layers.Conv2D(32, (25,25), activation='relu', input_shape=(28, 28, 1)))
model25x25.add(layers.Flatten())
model25x25.add(layers.Dense(250, activation='relu'))
model25x25.add(layers.Dense(10, activation='softmax'))

model25x25.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 4, 4, 32)          20032     
                                                                 
 flatten_4 (Flatten)         (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 250)               128250    
                                                                 
 dense_9 (Dense)             (None, 10)                2510      
                                                                 
Total params: 150792 (589.03 KB)
Trainable params: 150792 (589.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
treinar(model25x25)

Epoch 1/5
938/938 [==============================] - 4s 3ms/step - loss: 0.2375 - accuracy: 0.9271
Epoch 2/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0977 - accuracy: 0.9699
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0717 - accuracy: 0.9788
Epoch 4/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0571 - accuracy: 0.9825
Epoch 5/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0487 - accuracy: 0.9857


In [21]:
stats(model25x25)

313/313 [==============================] - 1s 2ms/step - loss: 0.0760 - accuracy: 0.9801


0.9800999760627747

## Resultados parciais foram inconclusivos com apenas uma camada de convolução

- 3x3: treinamento extremamente lento, muitos parâmetros e bons resultados.
- 5x5: treinamento razoavelmente lento, muitos parâmetros e bons resultados.
- 7x7: treinamento em tempo aceitável, quantidade menor de parâmetros e melhores resultados ( lembrando que usamos apenas UMA camada ).
- 25x25: treinamento muito rápido, quantidade pequena de parâmetros e piores resultados, o que faz sentido pois são filtros 25x25 para imagens 28x28.

#

# Múltiplas camadas de convolução

### - 3 camadas de convolução 3x3

In [22]:
model_camadas_3 = models.Sequential()
model_camadas_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas_3.add(layers.Flatten())
model_camadas_3.add(layers.Dense(250, activation='relu'))
model_camadas_3.add(layers.Dense(10, activation='softmax'))

model_camadas_3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_8 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 conv2d_9 (Conv2D)           (None, 22, 22, 32)        9248      
                                                                 
 flatten_5 (Flatten)         (None, 15488)             0         
                                                                 
 dense_10 (Dense)            (None, 250)               3872250   
                                                                 
 dense_11 (Dense)            (None, 10)                2510      
                                                                 
Total params: 3893576 (14.85 MB)
Trainable params: 389

In [23]:
treinar(model_camadas_3)


Epoch 1/5
938/938 [==============================] - 7s 6ms/step - loss: 0.1247 - accuracy: 0.9617
Epoch 2/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0366 - accuracy: 0.9884
Epoch 3/5
938/938 [==============================] - 5s 6ms/step - loss: 0.0210 - accuracy: 0.9935
Epoch 4/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0135 - accuracy: 0.9958
Epoch 5/5
938/938 [==============================] - 5s 6ms/step - loss: 0.0086 - accuracy: 0.9974


In [24]:
stats(model_camadas_3)


313/313 [==============================] - 1s 3ms/step - loss: 0.0380 - accuracy: 0.9914


0.9914000034332275

---

In [25]:
model_camadas_8x8 = models.Sequential()
model_camadas_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1)))
model_camadas_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1)))
model_camadas_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1)))
model_camadas_8x8.add(layers.Flatten())
model_camadas_8x8.add(layers.Dense(250, activation='relu'))
model_camadas_8x8.add(layers.Dense(10, activation='softmax'))

model_camadas_8x8.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 21, 21, 32)        2080      
                                                                 
 conv2d_11 (Conv2D)          (None, 14, 14, 32)        65568     
                                                                 
 conv2d_12 (Conv2D)          (None, 7, 7, 32)          65568     
                                                                 
 flatten_6 (Flatten)         (None, 1568)              0         
                                                                 
 dense_12 (Dense)            (None, 250)               392250    
                                                                 
 dense_13 (Dense)            (None, 10)                2510      
                                                                 
Total params: 527976 (2.01 MB)
Trainable params: 52797

In [26]:
treinar(model_camadas_8x8)

Epoch 1/5
938/938 [==============================] - 6s 5ms/step - loss: 0.1591 - accuracy: 0.9493
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0505 - accuracy: 0.9854
Epoch 3/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0366 - accuracy: 0.9898
Epoch 4/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0308 - accuracy: 0.9912
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0261 - accuracy: 0.9927


In [27]:
stats(model_camadas_8x8)

313/313 [==============================] - 1s 3ms/step - loss: 0.0331 - accuracy: 0.9918


0.9918000102043152

## - Testando a utilização de padding no teste 8x8

In [28]:
model_camadas_padding_8x8 = models.Sequential()
model_camadas_padding_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8x8.add(layers.Flatten())
model_camadas_padding_8x8.add(layers.Dense(250, activation='relu'))
model_camadas_padding_8x8.add(layers.Dense(10, activation='softmax'))

model_camadas_padding_8x8.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 28, 28, 32)        2080      
                                                                 
 conv2d_14 (Conv2D)          (None, 28, 28, 32)        65568     
                                                                 
 conv2d_15 (Conv2D)          (None, 28, 28, 32)        65568     
                                                                 
 flatten_7 (Flatten)         (None, 25088)             0         
                                                                 
 dense_14 (Dense)            (None, 250)               6272250   
                                                                 
 dense_15 (Dense)            (None, 10)                2510      
                                                                 
Total params: 6407976 (24.44 MB)
Trainable params: 640

In [29]:
treinar(model_camadas_padding_8x8)

Epoch 1/5
938/938 [==============================] - 14s 13ms/step - loss: 0.1468 - accuracy: 0.9540
Epoch 2/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0421 - accuracy: 0.9872
Epoch 3/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0294 - accuracy: 0.9916
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0219 - accuracy: 0.9934
Epoch 5/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0175 - accuracy: 0.9952


# Resultados sobre o uso de padding

O número de parâmetros cresceu exponencialmente, algo que faz sentido devido ao fato de que o "shape" do output não estava decrescendo, porém nesse caso a accuracy ainda caiu, ou seja, o padding não foi tão interessante assim para nosso projeto.

- 8x8 sem padding: 0.9919000267982483
- 8x8 com padding: 0.9914000034332275


In [30]:
stats(model_camadas_padding_8x8)

313/313 [==============================] - 1s 4ms/step - loss: 0.0391 - accuracy: 0.9903


0.9902999997138977

# ???????????????????????????????????
# ????????????????????????????????????????????????

In [31]:

model_camadas8_3 = models.Sequential()
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Flatten())
model_camadas8_3.add(layers.Dense(250, activation='relu'))
model_camadas8_3.add(layers.Dense(10, activation='softmax'))

model_camadas8_3.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 conv2d_17 (Conv2D)          (None, 24, 24, 32)        9248      
                                                                 
 conv2d_18 (Conv2D)          (None, 22, 22, 32)        9248      
                                                                 
 conv2d_19 (Conv2D)          (None, 20, 20, 32)        9248      
                                                                 
 conv2d_20 (Conv2D)          (None, 18, 18, 32)        9248      
                                                                 
 conv2d_21 (Conv2D)          (None, 16, 16, 32)        9248      
                                                                 
 conv2d_22 (Conv2D)          (None, 14, 14, 32)       

In [32]:
treinar(model_camadas8_3)

Epoch 1/5
938/938 [==============================] - 11s 9ms/step - loss: 0.1766 - accuracy: 0.9446
Epoch 2/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0503 - accuracy: 0.9852
Epoch 3/5
938/938 [==============================] - 9s 9ms/step - loss: 0.0347 - accuracy: 0.9895
Epoch 4/5
938/938 [==============================] - 8s 8ms/step - loss: 0.0273 - accuracy: 0.9916
Epoch 5/5
938/938 [==============================] - 7s 8ms/step - loss: 0.0228 - accuracy: 0.9930


In [33]:
stats(model_camadas8_3)

313/313 [==============================] - 1s 3ms/step - loss: 0.0323 - accuracy: 0.9907


0.9907000064849854

# Utilizando a mesma rede para outro teste com padding

Mais uma vez resultados não tão interessantes utilizando padding


In [34]:
model_camadas_padding_8_3 = models.Sequential()
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Flatten())
model_camadas_padding_8_3.add(layers.Dense(250, activation='relu'))
model_camadas_padding_8_3.add(layers.Dense(10, activation='softmax'))

model_camadas_padding_8_3.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 conv2d_25 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 conv2d_26 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 conv2d_27 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 conv2d_28 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 conv2d_29 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 conv2d_30 (Conv2D)          (None, 28, 28, 32)       

In [35]:
treinar(model_camadas_padding_8_3)

Epoch 1/5
938/938 [==============================] - 14s 12ms/step - loss: 0.1567 - accuracy: 0.9522
Epoch 2/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0422 - accuracy: 0.9871
Epoch 3/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0273 - accuracy: 0.9915
Epoch 4/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0182 - accuracy: 0.9943
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0137 - accuracy: 0.9960


In [36]:
stats(model_camadas_padding_8_3)

313/313 [==============================] - 1s 3ms/step - loss: 0.0342 - accuracy: 0.9890


0.9890000224113464

# Testando a utilização de stride

observe o resultado do sumário da rede neural, a quantidade de parâmetros pequena afeta diretamente o tempo de treinamento e o resultado.

In [37]:
model_camadas_stride_3x3 = models.Sequential()
model_camadas_stride_3x3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1), strides=(2,2)))
model_camadas_stride_3x3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1), strides=(2,2)))
model_camadas_stride_3x3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1), strides=(2,2)))
model_camadas_stride_3x3.add(layers.Flatten())
model_camadas_stride_3x3.add(layers.Dense(250, activation='relu'))
model_camadas_stride_3x3.add(layers.Dense(10, activation='softmax'))
model_camadas_stride_3x3.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 13, 13, 32)        320       
                                                                 
 conv2d_33 (Conv2D)          (None, 6, 6, 32)          9248      
                                                                 
 conv2d_34 (Conv2D)          (None, 2, 2, 32)          9248      
                                                                 
 flatten_10 (Flatten)        (None, 128)               0         
                                                                 
 dense_20 (Dense)            (None, 250)               32250     
                                                                 
 dense_21 (Dense)            (None, 10)                2510      
                                                                 
Total params: 53576 (209.28 KB)
Trainable params: 535

In [38]:
treinar(model_camadas_stride_3x3)

Epoch 1/5
938/938 [==============================] - 6s 5ms/step - loss: 0.2667 - accuracy: 0.9194
Epoch 2/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0890 - accuracy: 0.9732
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0613 - accuracy: 0.9814
Epoch 4/5
938/938 [==============================] - 4s 5ms/step - loss: 0.0469 - accuracy: 0.9854
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0381 - accuracy: 0.9884


In [39]:
stats(model_camadas_stride_3x3)

313/313 [==============================] - 1s 3ms/step - loss: 0.0448 - accuracy: 0.9869


0.9868999719619751

# Testes variando o Pooling
Nós vamos agora variar os poolings, entre diferentes tipos, como MaxPooling ou AvgPooling, e diferentes valores

## Testes com Average Pooling e o Max Pooling

### Max Pooling

In [41]:
model_max_pooling = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model_max_pooling.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_35 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 8, 8, 32)          0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 6, 6, 64)          18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 2, 2, 64)          0         
 g2D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 256)               0         
                                                                 
 dense_22 (Dense)            (None, 128)             

In [42]:
treinar(model_max_pooling)

Epoch 1/5
938/938 [==============================] - 7s 5ms/step - loss: 0.3683 - accuracy: 0.8886
Epoch 2/5
938/938 [==============================] - 4s 5ms/step - loss: 0.1128 - accuracy: 0.9653
Epoch 3/5
938/938 [==============================] - 4s 5ms/step - loss: 0.0788 - accuracy: 0.9766
Epoch 4/5
938/938 [==============================] - 4s 5ms/step - loss: 0.0625 - accuracy: 0.9810
Epoch 5/5
938/938 [==============================] - 4s 5ms/step - loss: 0.0535 - accuracy: 0.9839


In [43]:
stats(model_max_pooling)

313/313 [==============================] - 1s 3ms/step - loss: 0.0326 - accuracy: 0.9890


0.9890000224113464

### Average Pooling

In [44]:
model_avg_pooling = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.AveragePooling2D(pool_size=(3, 3)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.AveragePooling2D(pool_size=(3, 3)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model_avg_pooling.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_37 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 average_pooling2d (Average  (None, 8, 8, 32)          0         
 Pooling2D)                                                      
                                                                 
 conv2d_38 (Conv2D)          (None, 6, 6, 64)          18496     
                                                                 
 average_pooling2d_1 (Avera  (None, 2, 2, 64)          0         
 gePooling2D)                                                    
                                                                 
 flatten_12 (Flatten)        (None, 256)               0         
                                                                 
 dense_24 (Dense)            (None, 128)             

In [46]:
treinar(model_avg_pooling)

Epoch 1/5
938/938 [==============================] - 5s 4ms/step - loss: 0.5579 - accuracy: 0.8234
Epoch 2/5
938/938 [==============================] - 5s 5ms/step - loss: 0.2143 - accuracy: 0.9358
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.1544 - accuracy: 0.9539
Epoch 4/5
938/938 [==============================] - 4s 4ms/step - loss: 0.1251 - accuracy: 0.9628
Epoch 5/5
938/938 [==============================] - 5s 5ms/step - loss: 0.1026 - accuracy: 0.9693


In [47]:
stats(model_avg_pooling)

313/313 [==============================] - 1s 4ms/step - loss: 0.0554 - accuracy: 0.9804


0.980400025844574

## Testes com diferentes valores para o Max Pooling
Como o Max Pooling demonstrou nos testes ser um pouco superior ao Average Pooling, vamos utilizarelepara tesatr com diferentes valores de pooling

### Max Pooling 2x2

In [48]:
model_max_pooling2 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model_max_pooling2.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_40 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_13 (Flatten)        (None, 1600)              0         
                                                                 
 dense_26 (Dense)            (None, 128)             

In [49]:
treinar(model_max_pooling2)

Epoch 1/5
938/938 [==============================] - 5s 4ms/step - loss: 0.2132 - accuracy: 0.9343
Epoch 2/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0720 - accuracy: 0.9788
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0541 - accuracy: 0.9839
Epoch 4/5
938/938 [==============================] - 4s 5ms/step - loss: 0.0447 - accuracy: 0.9865
Epoch 5/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0395 - accuracy: 0.9887


In [50]:
stats(model_max_pooling2)

313/313 [==============================] - 1s 3ms/step - loss: 0.0264 - accuracy: 0.9910


0.9909999966621399

### Max Pooling 5x5

In [56]:
model_max_pooling5 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(5, 5)),  # Reduz para 5x5
    layers.Conv2D(filters=64, kernel_size=(1, 1), activation='relu'),  # Usa kernel 1x1 para não alterar a dimensão
    layers.MaxPooling2D(pool_size=(5, 5)),  # Reduz para 1x1
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model_max_pooling5.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_53 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 5, 5, 32)          0         
 ng2D)                                                           
                                                                 
 conv2d_54 (Conv2D)          (None, 5, 5, 64)          2112      
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 1, 1, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_19 (Flatten)        (None, 64)                0         
                                                                 
 dense_38 (Dense)            (None, 128)             

In [57]:
treinar(model_max_pooling5)

Epoch 1/5
938/938 [==============================] - 6s 5ms/step - loss: 1.5611 - accuracy: 0.4550
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.9614 - accuracy: 0.6739
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.7498 - accuracy: 0.7522
Epoch 4/5
938/938 [==============================] - 4s 5ms/step - loss: 0.6114 - accuracy: 0.8018
Epoch 5/5
938/938 [==============================] - 3s 4ms/step - loss: 0.5289 - accuracy: 0.8306


In [58]:
stats(model_max_pooling5)

313/313 [==============================] - 1s 3ms/step - loss: 0.3668 - accuracy: 0.8836


0.8835999965667725

# Testes de Dropout

## Sem Dropout

In [59]:
model_no_dropout = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.0),
    layers.Dense(10, activation='softmax')
])
model_no_dropout.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_55 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 13, 13, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_56 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 5, 5, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_20 (Flatten)        (None, 1600)              0         
                                                                 
 dense_40 (Dense)            (None, 128)             

In [60]:
treinar(model_no_dropout)

Epoch 1/5
938/938 [==============================] - 5s 5ms/step - loss: 0.1578 - accuracy: 0.9509
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0469 - accuracy: 0.9853
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0322 - accuracy: 0.9898
Epoch 4/5
938/938 [==============================] - 4s 5ms/step - loss: 0.0236 - accuracy: 0.9928
Epoch 5/5
938/938 [==============================] - 8s 9ms/step - loss: 0.0173 - accuracy: 0.9946


In [61]:
stats(model_no_dropout)

313/313 [==============================] - 2s 5ms/step - loss: 0.0248 - accuracy: 0.9920


0.9919999837875366

## Dropout Baixo (0.1)

In [62]:
model_dropout_0_1 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(10, activation='softmax')
])
model_dropout_0_1.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_57 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 13, 13, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_58 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 5, 5, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_21 (Flatten)        (None, 1600)              0         
                                                                 
 dense_42 (Dense)            (None, 128)             

In [63]:
treinar(model_dropout_0_1)

Epoch 1/5
938/938 [==============================] - 5s 4ms/step - loss: 0.1654 - accuracy: 0.9488
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0504 - accuracy: 0.9845
Epoch 3/5
938/938 [==============================] - 4s 5ms/step - loss: 0.0347 - accuracy: 0.9891
Epoch 4/5
938/938 [==============================] - 4s 5ms/step - loss: 0.0269 - accuracy: 0.9916
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0201 - accuracy: 0.9935


In [64]:
stats(model_dropout_0_1)

313/313 [==============================] - 1s 3ms/step - loss: 0.0284 - accuracy: 0.9912


0.9911999702453613

## Dropout Alto (0.5)

In [65]:
model_dropout_0_5 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model_dropout_0_5.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_59 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 13, 13, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_60 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 5, 5, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_22 (Flatten)        (None, 1600)              0         
                                                                 
 dense_44 (Dense)            (None, 128)             

In [66]:
treinar(model_dropout_0_5)

Epoch 1/5
938/938 [==============================] - 52s 4ms/step - loss: 0.2282 - accuracy: 0.9307
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0746 - accuracy: 0.9780
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0558 - accuracy: 0.9835
Epoch 4/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0448 - accuracy: 0.9868
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0397 - accuracy: 0.9885


In [67]:
stats(model_dropout_0_5)

313/313 [==============================] - 1s 2ms/step - loss: 0.0273 - accuracy: 0.9916


0.991599977016449